In [5]:
from rdflib import Graph, URIRef, Namespace, Literal, XSD, RDF
from rdflib.plugins.stores.sparqlstore import SPARQLStore
from tqdm.autonotebook import tqdm
import pandas as pd
from copy import deepcopy
import numpy
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict
from rapidfuzz import process, fuzz, distance
import numpy as np
import math

import torch
from transformers import BertTokenizer
from transformers import BertModel
from sklearn.metrics.pairwise import cosine_similarity

import concurrent.futures

import matplotlib.pyplot as plt

import numbers

import dateutil.parser as dparser
import datetime

/var/folders/6f/k2t20zbs4db5khp6mgdq_06m0000gn/T/ipykernel_46746/3205090218.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [44]:
GRAPH_1 = "/Users/yamamotobikutorueiichi/codes/custom_paris/memoryalpha.xml"
GRAPH_1_INV_FUNC_PATH = '/Users/yamamotobikutorueiichi/codes/custom_paris/memoryalpha-inversability.csv'

GRAPH_2 = "/Users/yamamotobikutorueiichi/codes/custom_paris/memory-beta.xml"
GRAPH_2_INV_FUNC_PATH = '/Users/yamamotobikutorueiichi/codes/custom_paris/memory-beta-inversability.csv'

GRAPH_3 = "/Users/yamamotobikutorueiichi/codes/custom_paris/startrek.ttl"
GRAPH_3_INV_FUNC_PATH = '/Users/yamamotobikutorueiichi/codes/custom_paris/startrek-inversability.csv'

#GRAPH_2 = "http://localhost:8890/sparql"
SUBJECT = 'subject'
PREDICATE = 'predicate'
INVERSE_FUNCTIONALITY = 'inverse_functionality'
INVERSABILITY = 'inversability'
MAX_LENGTH_FULL_MATCH = 10000
LABEL_PREDICATE = "http://www.w3.org/2000/01/rdf-schema#label"
FILE_FOLDER = ""
FILE_FOLDER_1_2 = "/Users/yamamotobikutorueiichi/codes/custom_paris/ma-mb/"
FILE_FOLDER_1_3 = "/Users/yamamotobikutorueiichi/codes/custom_paris/ma-se/"
FILE_FOLDER_2_3 = "/Users/yamamotobikutorueiichi/codes/custom_paris/mb-se/"
FILE_FOLDER_LIST = [FILE_FOLDER_1_2, FILE_FOLDER_1_3, FILE_FOLDER_2_3]
GRAPH_1_ABSTRACT = "http://dbkwik.webdatacommons.org/ontology/abstract"
GRAPH_2_ABSTRACT = "http://dbkwik.webdatacommons.org/ontology/abstract"
CATEGORICAL = 'categorical'
DATASET = ''
GENERAL_THRESHOLD = 0.1

In [45]:
def load_graph(GRAPH, INV_FUNC_PATH):
    graph = Graph()
    graph.parse(GRAPH)
    
    inv_func_df = pd.read_csv(INV_FUNC_PATH)
    
    inv_func_dict = dict()
    
    for _, row in inv_func_df.iterrows():
        inv_func_dict[row['predicate']] = row['functionality']
    
    graph_info_dict = {
        'graph': graph,
        'inv_func_dict' : inv_func_dict
    }
    
    return graph_info_dict

In [46]:
graph_info_dict_1 = load_graph(GRAPH_1, GRAPH_1_INV_FUNC_PATH)

In [28]:
graph_info_dict_2 = load_graph(GRAPH_2, GRAPH_2_INV_FUNC_PATH)

In [29]:
graph_info_dict_3 = load_graph(GRAPH_3, GRAPH_3_INV_FUNC_PATH)

In [47]:
def load_mappings(FILE_FOLDER, graph_info_dict_1, graph_info_dict_2):
    entity_sim_df = pd.read_csv(FILE_FOLDER + "entity_sim_9.csv")
    
    entity_sim_dict = dict()
    
    for _, row in entity_sim_df.iterrows():
        e1 = row['e1']
        e2 = row['e2']
        sim = row['sim']

        entity_sim_dict.setdefault(e1, {})[e2] = sim
    
    class_sim_df = pd.read_csv(FILE_FOLDER + 'class_sem_prob.csv')

    for _, row in class_sim_df.iterrows():
        c1 = row['c1']
        c2 = row['c2']
        sim = row['sim']

        entity_sim_dict.setdefault(c1, {})[c2] = sim
    
    predicate_sim = pd.read_csv(FILE_FOLDER + 'predicate_sem_prob.csv')
    graph_1_func_dict = graph_info_dict_1['inv_func_dict']
    graph_2_func_dict = graph_info_dict_2['inv_func_dict']
    
    predicate_sim['mul_sim'] = predicate_sim.apply(lambda x: x['sim'] * graph_1_func_dict[x['p1']] * graph_2_func_dict[x['p2']], axis=1)

    predicate_sim = predicate_sim.drop(index=0)
    
    mapping_info_dict = {
        'entity_sim_dict': entity_sim_dict,
        'predicate_sim': predicate_sim
    }
    
    return mapping_info_dict

In [48]:
mapping_info_dict_1_2 = load_mappings(FILE_FOLDER_1_2, graph_info_dict_1, graph_info_dict_2)

In [49]:
mapping_info_dict_1_3 = load_mappings(FILE_FOLDER_1_3, graph_info_dict_1, graph_info_dict_3)

In [50]:
mapping_info_dict_2_3 = load_mappings(FILE_FOLDER_2_3, graph_info_dict_2, graph_info_dict_3)

In [51]:
graph_info_dict_1['mapping_1'] = [mapping_info_dict_1_2, mapping_info_dict_1_3]

graph_info_dict_2['mapping_1'] = [mapping_info_dict_2_3]
graph_info_dict_2['mapping_2'] = [mapping_info_dict_1_2]

graph_info_dict_3['mapping_2'] = [mapping_info_dict_1_3, mapping_info_dict_2_3]

In [52]:
def get_types(graph, pred):
    graph = graph_info_dict['graph']
    pred = graph_info_dict['']
    pred_uri = URIRef(pred)
    type_dict = dict()
    
    for s, o in graph.subject_objects(pred_uri):
        o_type = None
        if type(o) == Literal:
            o_type = type(o.toPython())
        else:
            o_type = type(o)
        
        if not o_type in type_dict:
            type_dict[o_type] = 0
        type_dict[o_type] += 1
        
    return type_dict

In [ ]:
graph_info_dict_1['predicate_types'] = get_types(graph_info_dict_1)